In [25]:
import re
import numpy as np
import xarray as xr
import cngi.conversion as conversion
import zarr

In [18]:
ignore = ['CALDEVICE', 'FEED', 'FIELD',
              'FLAG_CMD', 'HISTORY', 'OBSERVATION',
              'POLARIZATION', 'SOURCE', 'SPECTRAL_WINDOW',
              'STATE', 'SYSCAL', 'SYSPOWER', 'WEATHER']

#msdx = conversion.convert_ms(infile='evla_bctest.ms', ignore=ignore)
msdx = zarr.convenience.load('evla_bctest.ms.zarr')
msdx

<LazyLoader: >

In [7]:
main = msdx.xds0
pointing = msdx.POINTING

In [8]:
def _compute_antenna_combinations(antenna1:'numpy.ndarray', antenna2:'numpy.ndarray')->'numpy.ndarray':
    """

    Parameters
    ----------
    antenna1
    antenna2

    Returns
    -------

    """

    baselines = np.array([])
    for left in antenna1:
        antennas = antenna2[antenna2 > left]
        for right in antennas:
            baselines = np.append(baselines, [left, right])

    return baselines.reshape(baselines.shape[0] // 2, 2)

In [9]:
main

<xarray.Dataset>
Dimensions:         (baseline: 6, time: 311, chan: 64, pol: 4, uvw_index: 3,
                     pol_id: 1, spw_id: 1)
Coordinates:
  * baseline        (baseline) int64 0 1 2 3 4 5
  * chan            (chan) float64 8.5e+09 8.502e+09 ... 8.624e+09 8.626e+09
    chan_width      (chan) float64 dask.array<chunksize=(32,), meta=np.ndarray>
    effective_bw    (chan) float64 dask.array<chunksize=(32,), meta=np.ndarray>
  * pol             (pol) int64 5 6 7 8
  * pol_id          (pol_id) int64 0
    resolution      (chan) float64 dask.array<chunksize=(32,), meta=np.ndarray>
  * spw_id          (spw_id) int64 0
  * time            (time) datetime64[ns] 2022-05-12T21:16:23.500000 ... 2022...
Dimensions without coordinates: uvw_index
Data variables: (12/17)
    ANTENNA1        (baseline) int64 dask.array<chunksize=(6,), meta=np.ndarray>
    ANTENNA2        (baseline) int64 dask.array<chunksize=(6,), meta=np.ndarray>
    ARRAY_ID        (time, baseline) int64 dask.array<chunksize=(100, 6), meta=np.ndarray>
    DATA            (time, baseline, chan, pol) complex128 dask.array<chunksize=(100, 6, 32, 1), meta=np.ndarray>
    DATA_WEIGHT     (time, baseline, chan, pol) float64 dask.array<chunksize=(100, 6, 32, 1), meta=np.ndarray>
    EXPOSURE        (time, baseline) float64 dask.array<chunksize=(100, 6), meta=np.ndarray>
    ...              ...
    OBSERVATION_ID  (time, baseline) int64 dask.array<chunksize=(100, 6), meta=np.ndarray>
    PROCESSOR_ID    (time, baseline) int64 dask.array<chunksize=(100, 6), meta=np.ndarray>
    SCAN_NUMBER     (time, baseline) int64 dask.array<chunksize=(100, 6), meta=np.ndarray>
    STATE_ID        (time, baseline) int64 dask.array<chunksize=(100, 6), meta=np.ndarray>
    TIME_CENTROID   (time, baseline) float64 dask.array<chunksize=(100, 6), meta=np.ndarray>
    UVW             (time, baseline, uvw_index) float64 dask.array<chunksize=(100, 6, 3), meta=np.ndarray>
Attributes: (12/16)
    bbc_no:               12
    corr_product:         [[0, 0], [0, 1], [1, 0], [1, 1]]
    data_groups:          [{'0': {'data': 'DATA', 'flag': 'FLAG', 'id': '0', ...
    freq_group:           0
    freq_group_name:      
    if_conv_chain:        0
    ...                   ...
    num_corr:             4
    ref_frequency:        8500000000.0
    sdm_corr_bit:         BITS_4x4
    sdm_num_bin:          1
    sdm_window_function:  UNIFORM
    total_bandwidth:      128000000.0

In [10]:
ant1 = main.ANTENNA1.data.compute()
ant1

array([0, 0, 0, 1, 1, 2])

In [11]:
ant2 = main.ANTENNA2.data.compute()
ant2

array([1, 2, 3, 2, 3, 3])

In [21]:
for i, j in enumerate(zip(ant1, ant2)):
    print('{0} {1}'.format(i, j))

0 (0, 1)
1 (0, 2)
2 (0, 3)
3 (1, 2)
4 (1, 3)
5 (2, 3)


In [38]:
a = {}
a[0] = (1,2)
a[1] = (3,4)

In [22]:
windows = ['xdsa0', 'xdsa1', 'xsa0', 'xds1']

In [31]:
for key in msdx.attrs.keys():
    match = re.match(r"xds[0-9]+", key)
    if match is not None:
        spw.append(match[0])
        
spw

AttributeError: 'LazyLoader' object has no attribute 'attrs'